In [0]:
DEBUG = True

GALAXY = 'fornax'

# Plot the target lists

Load the target lists and plot them. The file names are looked up from the netflow
configuration file under `./configs/netflow`.

In [0]:
import os, sys
from datetime import datetime
import numpy as np
import pandas as pd
import matplotlib as mpl
import matplotlib.pyplot as plt
from matplotlib.gridspec import GridSpec
from matplotlib.colors import LinearSegmentedColormap
from matplotlib.patches import Ellipse, Circle
from matplotlib.gridspec import GridSpec
from scipy.special import logsumexp
from scipy.interpolate import interp1d

from astropy import wcs
from astropy import units as u
from astropy.coordinates import Angle, SkyCoord
from astropy.time import Time

In [0]:
plt.rc('font', size=6) #controls default text size

In [0]:
%load_ext autoreload
%autoreload 2

In [0]:
if DEBUG and 'debug' not in globals():
    import debugpy
    debugpy.listen(('0.0.0.0', int(os.environ['PFS_TARGETING_DEBUGPORT'])))
    debug = True

# Plot definitions

In [0]:
import pfs.utils
from pfs.ga.targeting.targets.dsph import *
from pfs.ga.targeting.instrument import *
from pfs.ga.targeting.diagram import CMD, CCD, FOV, FP, ColorAxis, MagnitudeAxis
from pfs.ga.targeting.photometry import Photometry, Magnitude, Color
from pfs.ga.targeting.projection import WcsProjection, Pointing
from pfs.ga.targeting.netflow import Netflow
from pfs.ga.targeting.io import DataFrameSerializer

In [0]:
galaxy = GALAXIES[GALAXY]
hsc = galaxy.get_photometry()
cmd = galaxy.get_cmd()
ccd = galaxy.get_ccd()
gaia_cmd = galaxy.get_cmd(Gaia)

In [0]:
pointings = galaxy.get_pointings(SubaruPFI)
pointing = pointings[0]

for p in pointings:
    print(p.ra, p.dec, p.posang)

In [0]:
wcs = WcsProjection(pointing, proj='TAN')
wfc = SubaruWFC(pointing)
fov = FOV(projection=wcs)
fp = FP(wfc)

## Load the target lists

In [0]:
from pfs.ga.targeting.config import NetflowConfig
from pfs.ga.targeting.scripts.netflowscript import NetflowScript
from pfs.ga.targeting.data import Catalog, Observation

In [0]:
# Put together the netflow configuration. Get the default config and add galaxy-specific
# settings, then load the target list configuration and apply it.

# Here we instantiate the command-line script class because it has all features
# to work with the target lists.

config = f'../../../configs/netflow/dSph/{GALAXY}/targets.py'
print(os.path.abspath(config))

args = {
    'dsph': GALAXY,
    'config': [ config ],
}

script = NetflowScript()
script._init_from_args(args)

netflow_config = script.config

In [0]:
for name, target_list in netflow_config.targets.items():
    print(name, target_list.prefix, target_list.path)

In [0]:
target_lists = script.load_source_target_lists()

In [0]:
cmap = plt.get_cmap('tab10')
colors = { 'sci': 'lightgray', 'cal': 'red', 'sky': cmap(2) }
size = { 'sci': 0.5, 'cal': 2, 'sky': 0.1 }

f = plt.figure(figsize=(8, 3), dpi=240)
gs = f.add_gridspec(1, 3, width_ratios=[2, 2, 3], wspace=0.4)

ax = f.add_subplot(gs[0])
for name, options in script._NetflowScript__config.targets.items():
    if options.prefix in ['sci', 'cal']:
        if target_lists[name].has_diagram_values(cmd, observed=True):
            cmd.plot_observation(ax, target_lists[name], c=colors[options.prefix], observed=True)

ax = f.add_subplot(gs[1])
for name, options in script._NetflowScript__config.targets.items():
    if options.prefix in ['sci', 'cal']:
        if target_lists[name].has_diagram_values(ccd, observed=True):
            ccd.plot_observation(ax, target_lists[name], c=colors[options.prefix], observed=True)

ax = f.add_subplot(gs[2], projection=wcs.wcs)
for name, options in script._NetflowScript__config.targets.items():
    if options.prefix in ['sci', 'cal']:
        fov.plot_observation(ax, target_lists[name], c=colors[options.prefix], size=size[options.prefix], observed=True)

f.suptitle('Stars selected based on color cuts.')

In [0]:
f = plt.figure(figsize=(3, 3), dpi=240)
gs = f.add_gridspec(1, 1)

ax = f.add_subplot(gs[0, 0], projection=wcs.wcs)
for name, options in script._NetflowScript__config.targets.items():
    if options.prefix in ['sky']:
        fov.plot_observation(ax, target_lists[name], c=colors[options.prefix], size=size[options.prefix], observed=True)

f.suptitle('Available sky positions')